# Advent of Code Day 3
https://adventofcode.com/2021/day/3

## Part 1 - Compute Power Consumption

In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
def gamma_binary(diagnostics: pd.DataFrame) -> str:
    """
    gamma is the least common values in each position, i.e. mode.
    """
    return "".join(diagnostics.mode().loc[0].to_list())

def epsilon_binary(diagnostics: pd.DataFrame) -> str:
    """
    epsilon is the least common values in each position, i.e. inversion of gamma.
    """
    return ''.join('1' if x == '0' else '0' for x in gamma_binary(diagnostics))

def power_consumption(diagnostics: pd.DataFrame) -> int:
    """
    computes power consumption by multiplying gamma and epsilon and converting to base 10.
    """
    return int(gamma_binary(diagnostics), base=2) * int(epsilon_binary(diagnostics), base=2)

In [3]:
# Unit testing our gamma and epsilon functions with the test data provided 
# the example.
bits_test = pd.DataFrame(
    [
        '00100', '11110', '10110', '10111', '10101', 
        '01111', '00111', '11100', '10000', '11001', 
        '00010', '01010'
    ], 
    columns=['Diag']
).Diag.str.split("",expand=True)\
    .replace("", np.nan, inplace=False)\
    .dropna(axis=1, how='all', inplace=False)\
    .dropna(axis=0, how='all', inplace=False)

assert gamma_binary(bits_test) == '10110'
assert epsilon_binary(bits_test) == '01001'

In [4]:
# Read our input.
data_directory: str  = 'data/day_3'
input_file = os.path.join(data_directory, 'input.txt')
df = pd.read_csv(input_file, header=None, names=['Diag'], dtype={'Diag':str})

# Convert to dataframe of bits.
bits = df.Diag.str.split("",expand=True)\
    .replace("", np.nan, inplace=False)\
    .dropna(axis=1, how='all', inplace=False)\
    .dropna(axis=0, how='all', inplace=False)
bits.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,1,0,0,1,1,0,0,1,0,1
1,0,1,0,1,0,0,0,1,1,1,0,0
2,1,0,0,0,0,0,1,1,0,0,0,1
3,0,0,1,1,1,1,1,1,0,1,0,1
4,1,0,0,0,1,0,1,1,0,1,0,1


In [5]:
# print(gamma_binary(bits))
# print(epsilon_binary(bits))

# Answer the question
print(power_consumption(bits))


3901196


## Part 2: Verifying the life support rating.
The life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

### Oxygen generator rating

1. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and **consider just the first bit** of those numbers.
2. Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
3. If you only have one number left, stop; this is the rating value for which you are searching.
4. Otherwise, repeat the process, considering the next bit to the right.


In [6]:
def oxygen_generator_rating_binary(diagnostics: pd.DataFrame) -> str:
    df = diagnostics.copy()
    
    for col in df.columns:
        s: pd.Series = df[col].value_counts()
        (mc, mc_cnt) = s.index[0], s.loc[s.index[0]]
        (lc, lc_cnt) = s.index[-1], s.loc[s.index[-1]]

        if mc_cnt == lc_cnt or mc == "1":
            df = df.drop(df[df[col].str.match("0")].index)
        else:
            df = df.drop(df[df[col].str.match("1")].index)

        if len(df.index) == 1:
            break

    return "".join(df.loc[df.index[0]].to_list())

def oxygen_generator_rating(diagnostics: pd.DataFrame) -> int:
    """
    Returns oxygen_generator_rating in decimal.
    """
    return int(oxygen_generator_rating_binary(diagnostics), base=2)

In [7]:
assert oxygen_generator_rating_binary(bits_test) == "10111"
assert oxygen_generator_rating(bits_test) == 23

### C02 Scrubber rating

1. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and **consider just the first bit** of those numbers.
2. Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.
3. If you only have one number left, stop; this is the rating value for which you are searching.
4. Otherwise, repeat the process, considering the next bit to the right.

In [8]:
def c02_scrubber_rating_binary(diagnostics: pd.DataFrame) -> str:
    """
    Returns c02_scrubber_rating in binary.
    """
    df = diagnostics.copy()
    
    for col in df.columns:
        s: pd.Series = df[col].value_counts()
        (mc, mc_cnt) = s.index[0], s.loc[s.index[0]]
        (lc, lc_cnt) = s.index[-1], s.loc[s.index[-1]]

        if mc_cnt == lc_cnt or lc == "0":
            df = df.drop(df[df[col].str.match("1")].index)
        else:
            df = df.drop(df[df[col].str.match("0")].index)

        if len(df.index) == 1:
            break

    return "".join(df.loc[df.index[0]].to_list())

def c02_scrubber_rating(diagnostics: pd.DataFrame) -> int:
    """
    Returns c02_scrubber_rating in decimal.
    """
    return int(c02_scrubber_rating_binary(diagnostics), base=2)

In [9]:
assert c02_scrubber_rating_binary(bits_test) == "01010"
assert c02_scrubber_rating(bits_test) == 10

### Life Support rating

Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

In [10]:
def life_support_rating(diagnostics: pd.DataFrame) -> int:
    """
    Compute life Support Rating.
    """
    return oxygen_generator_rating(diagnostics) * c02_scrubber_rating(diagnostics)

assert life_support_rating(bits_test) == 230

### Finding part 2's answer.

In [11]:
# Read our input.
data_directory: str  = 'data/day_3'
input_file = os.path.join(data_directory, 'input.txt')
df = pd.read_csv(input_file, header=None, names=['Diag'], dtype={'Diag':str})

# Convert to dataframe of bits.
bits = df.Diag.str.split("",expand=True)\
    .replace("", np.nan, inplace=False)\
    .dropna(axis=1, how='all', inplace=False)\
    .dropna(axis=0, how='all', inplace=False)

life_support_rating(bits)

4412188